### 도로표지판과 신호등 클래스를 담고 있는 파일만 선별

- 우리는 2번 클래스(도로표지판)과 3번 클래스(신호등)만 감지하는 모델을 만들 것입니다.

- 따라서, 2번 클래스와 3번 클래스 라벨이 없는 이미지는 사용할 필요가 없으므로 제외합니다.

- 단, 제외된 이미지들 중 일부는 모델 정확도를 높이고 오탐지(False Positive)를 줄이기 위한 Background image로 추후에 사용할 것입니다.

원천 데이터 파일이 저장된 폴더 구조를 다시 봅시다.

>&nbsp;&nbsp;<br>
>D드라이브<br>
>&nbsp;&nbsp;└ pgu_traffic_dataset<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ sources<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images → 이미지 파일(png) 18,481개<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ annotations → 라벨 파일(json) 18,481개<br>
>&nbsp;&nbsp;

In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm

In [ ]:
ann_path = "D:/pgu_traffic_dataset/sources/annotations"
ann_file_list = os.listdir(ann_path)

2번 혹은 3번 클래스 정보를 담고 있는 annotation 파일만 걸러내 selected에 파일명을 담아 놓습니다.

반복문을 도는 김에 2번 클래스의 개수와 3번 클래스의 개수를 계산해 데이터 균형이 맞는지 확인합니다.

In [ ]:
selected = [] #조건에 맞는 파일(2, 3번 클래스 정보를 담고 있는 파일)을 담아놓을 빈 리스트 생성
not_selected = [] #조건에 맞지 않는 파일을 담아놓을 빈 리스트 생성(background image로 사용)
class_2_cnt = 0 #2번 클래스 라벨 개수를 세기 위해 0으로 초기화된 class_2_cnt 변수 생성
class_3_cnt = 0 #3번 클래스 라벨 개수를 세기 위해 0으로 초기화된 class_3_cnt 변수 생성

for ann_file in tqdm(ann_file_list):
    ann_str = open("{}/{}".format(ann_path, ann_file), "r").read() #어노테이션 파일 로드
    ann = json.loads(ann_str) #어노테이션 파일의 json 문자열을 dict로 변환
    this_class_2_cnt = len([x for x in ann["annotations"] if x["category_id"] == 2]) #현재 파일의 2번 클래스 라벨 개수
    this_class_3_cnt = len([x for x in ann["annotations"] if x["category_id"] == 3]) #현재 파일의 3번 클래스 라벨 개수
    if this_class_2_cnt > 0 or this_class_3_cnt > 0: #2번, 3번 라벨 중 하나라도 있는 이미지라면
        selected.append(ann_file) #selected 리스트에 파일명을 append
    elif this_class_2_cnt == 0 and this_class_3_cnt == 0: #2번, 3번 라벨이 하나도 없는 이미지라면
        not_selected.append(ann_file) #추후 background image로 사용하기 위해 저장
    class_2_cnt += this_class_2_cnt #총 2번 클래스 라벨 개수 업데이트
    class_3_cnt += this_class_3_cnt #총 3번 클래스 라벨 개수 업데이트

print("2, 3번 클래스가 하나라도 포함된 이미지 : {}개".format(len(selected))) # -> 18481개 중 2, 3번 클래스가 포함된 이미지 17146개가 선택
print("2, 3번 클래스가 하나도 포함되지 않은 이미지 : {}개".format(len(not_selected))) # -> 2, 3번 클래스가 없는 이미지 1335개 → background image로 사용
print("2번 클래스 라벨의 개수 : {}개".format(class_2_cnt)) # -> 168391개
print("3번 클래스 라벨의 개수 : {}개".format(class_3_cnt)) # -> 82978개

클래스 별 개수는 균형이 맞아야 모델 성능이 좋아집니다. 지금처럼 개수 차이가 나는 상황에서는 3개의 선택지가 있습니다.

1. 3번 클래스가 포함되어 있는 이미지를 더 추가한다.

2. 2번 클래스가 포함되어 있는 이미지를 삭제한다.

3. 데이터 불균형이 있지만 그냥 진행한다.

1번이 모델 성능에 가장 좋겠으나, 추가할 이미지가 없고 데이터 양이 충분하므로 2번 방식으로 진행합니다.

In [ ]:
#2번 클래스가 포함된 이미지를 그냥 제거하면 3번 클래스의 개수도 줄어들게 됨
#따라서, 3번 클래스(신호등) 라벨보다 2번 클래스(표지판) 라벨이 1.7배 이상 많이 포함된 이미지일 경우에만 제거하는 것을 기준으로 하여 진행
to_remove = [] #제거할 이미지 파일명를 담아 놓을 빈 리스트 생성
class_2_cnt = 0
class_3_cnt = 0
for ann_file in tqdm(selected): #2, 3번 클래스를 하나라도 포함한 이미지 파일(17146개)만 루프
    ann_str = open("{}/{}".format(ann_path, ann_file), "r").read()
    ann = json.loads(ann_str)
    this_class_2_cnt = len([x for x in ann["annotations"] if x["category_id"] == 2])
    this_class_3_cnt = len([x for x in ann["annotations"] if x["category_id"] == 3])
    #2번 클래스 라벨이 3번 클래스 라벨보다 1.7배 더 많은 이미지라면 파일명을 to_remove에 append
    if this_class_3_cnt*1.7 <= this_class_2_cnt:
        to_remove.append(ann_file)
    else:
        class_2_cnt += this_class_2_cnt
        class_3_cnt += this_class_3_cnt
print("제거할 이미지 수 : {}".format(len(to_remove))) # -> 11137개
print("남은 이미지 수 : {}".format(len(selected) - len(to_remove))) # -> 6009개
print("2번 클래스 라벨의 개수 : {}".format(class_2_cnt)) # -> 56395개
print("3번 클래스 라벨의 개수 : {}".format(class_3_cnt)) # -> 53207개
# -> 6009개의 이미지만 남게 되었지만 2번 클래스 라벨과 3번 클래스 라벨의 개수 차이가 많이 줄어들었음

최종 선택된 파일 목록을 파일로 저장해 놓습니다.
백그라운드 이미지 또한 추후 사용할 것이므로 목록을 파일로 저장해 놓습니다.

In [ ]:
final_selected = [x for x in selected if x not in to_remove]
final_selected_json = json.dumps(final_selected)
not_selected_json = json.dumps(not_selected)
open("최종 선별된 파일 목록.json", "w", encoding="utf-8").write(final_selected_json)
open("백그라운드 파일 목록.json", "w", encoding="utf-8").write(not_selected_json)
len(final_selected), len(not_selected) # -> (6009, 1335)